# Forecasting - Facebook Prophet


https://facebook.github.io/prophet/

https://research.fb.com/blog/2017/02/prophet-forecasting-at-scale/

https://peerj.com/preprints/3190.pdf

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline
mpl.rcParams['figure.figsize'] = (16, 10)
pd.set_option('display.max_rows', 500)

import plotly.graph_objects as go

In [ ]:
from fbprophet import Prophet

In [ ]:
%matplotlib inline
plt.style.use('fivethirtyeight')

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return np.mean(np.abs((y_true-y_pred)/y_true))*100

# Trivial Forecast (Rolling Mean)

In [ ]:
df = pd.DataFrame({'X' : np.arange(1,10)})
df['Y'] = df.rolling(3).mean()

In [ ]:
df

# Small Data Set

In [ ]:
df_all = pd.read_csv('E:/ads_covid-19/data/processed/COVID_flat_small_table.csv',sep=';')

In [ ]:
df_all

In [ ]:
df = df_all[['Date', 'Germany']]

In [ ]:
df = df.rename(columns = {'Date': 'ds', 'Germany':'y'})

In [ ]:
df

In [ ]:
ax = df.set_index('ds').plot(figsize = (12,8), logy = True)
ax.set_ylabel('Daily Number of confimed cases')
ax.set_xlabel('Date')
plt.show()

In [ ]:
# Default uncertainty interval is 80%
# my_model = Prophet(interval_width=0.95)
my_model = Prophet(growth='logistic')  

In [ ]:
df['cap']=1000000. # 'cap' is required for logistic model
my_model.fit(df)

In [ ]:
# Define the periods and the frequency 'D'== days
future_dates = my_model.make_future_dataframe(periods = 10, freq = 'D')
future_dates['cap'] = 1000000.
future_dates.tail()

In [ ]:
forecast = my_model.predict(future_dates)

In [ ]:
my_model.plot(forecast, uncertainty=True);

In [ ]:
import plotly.offline as py
from fbprophet.plot import plot_plotly

fig = plot_plotly(my_model, forecast)
fig.update_layout(
    width=1024,
    height=900,
    xaxis_title="Time",
    yaxis_title="Confirmed infected people (source Johns Hopkins csse) (log-scale)",
)
fig.update_yaxes(type="log",range=[1.1,5.5])
#py.iplot(fig)
fig.show()

In [ ]:
my_model.plot_components(forecast);

In [ ]:
forecast[['ds', 'trend']].set_index('ds').plot(logy = True, figsize = (12,8))

# Cross Validation

In [ ]:
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics

In [ ]:
df_cv = cross_validation(my_model, 
                         initial='40 days', # 40 days for training
                         period='1 days',   # New prediction rn every day                     
                         horizon = '7 days') # Predict 7 days into future

In [ ]:
df_cv.sort_values(by = ['cutoff', 'ds'])[0:14]

In [ ]:
df_p = performance_metrics(df_cv)

In [ ]:
df_p

In [ ]:
from fbprophet.plot import plot_cross_validation_metric

In [ ]:
fig = plot_cross_validation_metric(df_cv, metric='mape')

# Diagonalplot

In [ ]:
horizon='7 days'
df_cv['horizon'] = df_cv.ds - df_cv.cutoff

In [ ]:
df_cv_7 = df_cv[df_cv['horizon']==horizon]

In [ ]:
df_cv_7

In [ ]:
y = df_cv_7.y
y_hat=df_cv[df_cv['horizon']==horizon]['yhat']

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(np.arange(max(y)), np.arange(max(y)),'--',label='diagonal')
ax.plot(y,y_hat,'-',label=horizon)
ax.set_title('Diagonal Plot')
ax.set_ylim(10, max(y))

ax.set_xlabel('truth: y')
ax.set_ylabel('prediciton: y_hat')
ax.set_yscale('log')

ax.set_xlim(10, max(y))
ax.set_xscale('log')
ax.legend(loc='best',
           prop={'size': 16});


# Trivial Forecast

In [ ]:
parse_dates=['Date']
df_all = pd.read_csv('E:/ads_covid-19/data/processed/COVID_flat_small_table.csv',sep=';', parse_dates=parse_dates)
df_trivial=df_all[['Date','Germany']]
df_trivial=df_trivial.rename(columns={'Date': 'ds',
                        'Germany': 'y'})
df_trivial['y_mean_r3']=df_trivial.y.rolling(3).mean()

df_trivial['cutoff']=df_trivial['ds'].shift(7)
df_trivial['y_hat']=df_trivial['y_mean_r3'].shift(7)
df_trivial['horizon']=df_trivial['ds']-df_trivial['cutoff']
print('MAPE: '+str(mean_absolute_percentage_error(df_trivial['y_hat'].iloc[12:,], df_trivial['y'].iloc[12:,])))
df_trivial